In [2]:
!wget https://www.wth.org/wp-content/uploads/2022/11/62-6010402_JMCGH_standardcharges.csv

--2023-04-13 19:35:07--  https://vhsemichigan.com/wp-content/uploads/sites/36/2022/08/261789242_Vibra-Hospital-of-Southeastern-Michigan_Standard-Charges.csv
Resolving vhsemichigan.com (vhsemichigan.com)... 161.35.12.166
Connecting to vhsemichigan.com (vhsemichigan.com)|161.35.12.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1106601 (1.1M) [text/csv]
Saving to: ‘261789242_Vibra-Hospital-of-Southeastern-Michigan_Standard-Charges.csv’

261789242_Vibra-Hos 100%[===================>]   1.05M  1.52MB/s    in 0.7s    

2023-04-13 19:35:08 (1.52 MB/s) - ‘261789242_Vibra-Hospital-of-Southeastern-Michigan_Standard-Charges.csv’ saved [1106601/1106601]



In [59]:
import polars as pl
df = pl.read_csv('csvs/62-6010402_JMCGH_standardcharges.csv', encoding = 'latin-1', skip_rows = 2, infer_schema_length = 0)

In [60]:
df.columns[:12]

['LINE TYPE',
 'CHARGE CODE/ PACKAGE',
 'CHARGE DESCRIPTION',
 'DRG',
 'APC',
 'CPT®/HCPCS',
 'ALTERNATE CPT®/HCPCS',
 'REV CODE',
 'MODIFIER',
 'NDC',
 'GROSS CHARGE',
 'DISCOUNTED CASH PRICE (UNINSURED)']

In [61]:
df = df.rename({
    'LINE TYPE':'code_type', 
    'CHARGE CODE/ PACKAGE':'internal_code',
    'CHARGE DESCRIPTION':'description',
    'DRG':'ms-drg', 
    'APC':'apc', 
    'CPT®/HCPCS':'hcpcs_cpt', 
    'ALTERNATE CPT®/HCPCS':'hcpcs_cpt_2', 
    'REV CODE':'rev_code', 
    'MODIFIER':'modifier', 
    'NDC':'ndc'
})

In [62]:
df.sample(20).to_pandas()

,code_type,internal_code,description,ms-drg,apc,hcpcs_cpt,hcpcs_cpt_2,rev_code,modifier,ndc,...,BCBS Blue Advantage All Plans,Cigna HealthSpring Medicare Advantage All Plans,Humana ChoiceCare All Plans,Humana Medicare Advantage All Plans,Amerigroup Tenncare All Plans,Amerigroup DSNP All Plans,Bluecare TennCare All Plans,Bluecare Plus DSNP All Plans,UHC Community Plan TennCare All Plans,UHC Community Plan Dual Complete DSNP All Plans
0,CDM,6411934,STENT VIATORR 10X7X2^PTB8107275,None,None,C1874,None,278,None,None,...,$0.00,$0.00,"$7,131.00",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
1,CDM,4510967,LAMELLAR BODY COUNT,None,None,83664,None,301,None,None,...,$0.00,$0.00,$64.50,$0.00,$0.00,$0.00,$7.13,$0.00,$16.02,$0.00
2,CDM,6986435,SCREW KREULOCK COMP SS 2.7X16MM--AR-8827CL-16,None,None,C1713,None,278,None,None,...,$0.00,$0.00,$442.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
3,CDM,6418589,SCREW LOCKING 5.0 X 70^212.223,None,None,C1713,None,278,None,None,...,$0.00,$0.00,$220.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
4,CPT,58270,VAG HYST W/ENTEROCELE REPAIR,None,5415,58270,None,360,None,None,...,"$4,394.11","$4,266.13","$7,701.75","$4,266.13","$3,626.21","$4,308.79","$2,259.48","$4,394.11","$1,638.00","$4,351.45"
5,CDM,6604221,COIL CONCERTO HELIX 8MM X 30CM^NV-8-30-HELIX,None,None,C1889,None,278,None,None,...,$0.00,$0.00,$700.50,$0.00,$0.00,$0.00,$467.00,$0.00,$0.00,$0.00
6,CDM,7509982-49560,naloxone 2 mg/ 2 mL injectable solution Syringe,None,None,None,None,250,None,76329-3369-01,...,$0.00,$0.00,$54.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
7,CDM,7501071-99574,Ipratropium Nasal 0.03% (21 mcg/inh) 30 ml Spray,None,None,A9270,None,250,None,24208-0398-30,...,$0.00,$0.00,$81.19,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
8,CDM,7509364-50027,iohexol 180 mg/mL Sol,None,None,Q9965,None,250,None,00407-1411-20,...,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
9,CPT,45990,SURG DX EXAM ANORECTAL,None,5313,45990,None,360,None,None,...,"$2,434.84","$2,363.92","$2,501.70","$2,363.92","$2,009.34","$2,387.56","$1,010.17","$2,434.84",$891.00,"$2,411.20"


In [63]:
df_cpt  = df.filter(pl.col('hcpcs_cpt').is_not_null()).drop(['hcpcs_cpt_2', 'ms-drg'])
df_cpt2 = df.filter(pl.col('hcpcs_cpt_2').is_not_null()).drop(['hcpcs_cpt', 'ms-drg'])
df_drg  = df.filter(pl.col('ms-drg').is_not_null()).drop(['hcpcs_cpt', 'hcpcs_cpt_2'])

In [64]:
payer_cols = df.columns[10:]

In [65]:
def melt_rates(df):
    id_vars = [c for c in df.columns if c not in payer_cols]
    value_vars = payer_cols
    variable_name = 'payer'
    value_name = 'rate'
    return df.melt(
        id_vars = id_vars,
        value_vars = value_vars,
        variable_name = variable_name,
        value_name = value_name
    )

In [67]:
df_cpt = melt_rates(df_cpt)
df_cpt2 = melt_rates(df_cpt2)
df_drg = melt_rates(df_drg)

In [69]:
df_cpt = df_cpt.rename({'hcpcs_cpt':'code'})
df_cpt2 = df_cpt2.rename({'hcpcs_cpt_2':'code'})
df_drg = df_drg.rename({'ms-drg':'code'})

In [71]:
df_cpt = df_cpt.with_columns(
    pl.lit('hcpcs_cpt').alias('code_prefix')
)

df_cpt2 = df_cpt2.with_columns(
    pl.lit('hcpcs_cpt').alias('code_prefix')
)

df_drg = df_drg.with_columns(
    pl.lit('ms-drg').alias('code_prefix')
)

In [75]:
df_cpt.columns

['code_type',
 'internal_code',
 'description',
 'apc',
 'code',
 'rev_code',
 'modifier',
 'ndc',
 'payer',
 'rate',
 'code_prefix']

In [76]:
df_cpt2 = df_cpt2.select(df_cpt.columns)
df_drg = df_drg.select(df_cpt.columns)

In [77]:
df = pl.concat([df_cpt, df_cpt2, df_drg])

In [91]:
df = df.filter(
    (pl.col('rate') != "Paid by Report") &
    (pl.col('rate') != "Not Billed to Insurance") &
    (pl.col('rate') != "Paid per CPT")
)

In [93]:
df = df.with_columns([
    (pl.when(pl.col('payer').str.contains('GROSS CHARGE')).then('gross')
     .when(pl.col('payer').str.contains('MINIMUM')).then('min')
     .when(pl.col('payer').str.contains('MAXIMUM')).then('max')
     .when(pl.col('payer').str.contains('CASH')).then('cash')
     .otherwise('payer')
    ).alias('payer_category'),
    (pl.when(pl.col('rate').str.contains('per day')).then('per day')
    ).otherwise(None)
    .alias('rate_method'),
    pl.col('rate').str.strip('$').str.replace(',', '').str.replace(' per day', '').cast(float),
    
])

In [94]:
df

code_type,internal_code,description,apc,code,rev_code,modifier,ndc,payer,rate,code_prefix,payer_category,rate_method
str,str,str,str,str,str,str,str,str,f64,str,str,str
"""CDM""","""7502394-17710""","""magnesium citr...",null,"""A9270""","""250""",null,"""50804-0693-38""","""GROSS CHARGE""",2.5,"""hcpcs_cpt""","""gross""",null
"""CDM""","""7508500-75089""","""polyethylene g...",null,"""A9270""","""250""",null,"""52268-0100-01""","""GROSS CHARGE""",43.75,"""hcpcs_cpt""","""gross""",null
"""CDM""","""7531048-2689""","""pramoxine-zinc...",null,"""A9270""","""250""",null,"""00904-7760-21""","""GROSS CHARGE""",3.49,"""hcpcs_cpt""","""gross""",null
"""CDM""","""7531039-2679""","""Cetaphil Lotio...",null,"""A9270""","""250""",null,"""00299-3918-08""","""GROSS CHARGE""",5.76,"""hcpcs_cpt""","""gross""",null
"""CDM""","""7503840-15488""","""mineral oil 10...",null,"""A9270""","""250""",null,"""54162-0190-16""","""GROSS CHARGE""",1.0,"""hcpcs_cpt""","""gross""",null
"""CDM""","""7507288-68601""","""Fleet (Sodium ...",null,"""A9270""","""250""",null,"""00536-7415-51""","""GROSS CHARGE""",4.0,"""hcpcs_cpt""","""gross""",null
"""CDM""","""7530186-1484""","""Lubriderm Unsc...",null,"""A9270""","""250""",null,"""52800-0488-26""","""GROSS CHARGE""",7.5,"""hcpcs_cpt""","""gross""",null
"""CDM""","""7501482-6991""","""mineral oil re...",null,"""A9270""","""250""",null,"""00132-0301-40""","""GROSS CHARGE""",4.75,"""hcpcs_cpt""","""gross""",null
"""CDM""","""7501378-48845""","""phenol 1.4% to...",null,"""A9270""","""250""",null,"""00536-2425-58""","""GROSS CHARGE""",5.51,"""hcpcs_cpt""","""gross""",null
